In [ ]:
directML = False

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast  # Used to convert string representation of a list to a list
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os
import seaborn as sns
from colour import Color
import glob
import sys

# Import necessary paths
absolute_root_path = os.path.abspath('').replace("\\","/") + "/.."
sys.path.insert(1,absolute_root_path)
from paths import preprocessed_datasets_path, save_weights_path,EMNIST_path

if not directML:
    from keras.models import Sequential, load_model, save_model
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization,Dropout,LSTM,Masking, Bidirectional
    from keras.optimizers import Adam
    from keras.utils import to_categorical
    from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger,Callback
    from keras import backend as K # to be able to change the learning rate on tge fly
else:
    from tensorflow.keras import backend as K # to be able to change the learning rate on tge fly
    from tensorflow.keras.models import Sequential, load_model, save_model
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization,Dropout,LSTM,Masking, Bidirectional
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger,Callback 

# Set parameters

In [ ]:
source_dataset = np.array(["DigiLeTs"]) #DigiLeTs or EMNIST or [DigiLeTs, BRUSH] or [DigiLeTs, EMNIST]
network_type = np.array(["LSTM"]) # LSTM or CNN
training_mode = "Fine-tuned" # "Normal" or "Fine-tuned"
image_shape = 28

In [ ]:
# return the distribution with a maximum number of classes
def return_subset(len_each_class, X,Y):
    unique_classes = np.unique(Y)

    selected_indices = []

    # Iterate over each unique class
    for class_label in unique_classes:
        # Find indices of examples belonging to the current class
        class_indices = np.where(Y == class_label)[0]
        
        # Select the first len_each_class examples for the current class
        selected_indices.extend(class_indices[:len_each_class])

    # Use the selected indices to extract the corresponding examples
    selected_data = X[selected_indices]
    selected_labels = Y[selected_indices]

    return selected_data,selected_labels

In [ ]:
# Allow for the use of an additional validation data during the training steps 
class AdditionalValidationSets(Callback):
    def __init__(self, validation_sets, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        for validation_set in self.validation_sets:
            if len(validation_set) not in [3, 4]:
                raise ValueError()
        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            if len(validation_set) == 3:
                validation_data, validation_targets, validation_set_name = validation_set
                sample_weights = None
            elif len(validation_set) == 4:
                validation_data, validation_targets, sample_weights, validation_set_name = validation_set
            else:
                raise ValueError()

            results = self.model.evaluate(x=validation_data,
                                          y=validation_targets,
                                          verbose=self.verbose,
                                          sample_weight=sample_weights,
                                          batch_size=self.batch_size)

            for metric, result in zip(self.model.metrics_names,results):
                valuename = validation_set_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

In [ ]:
# return the letter corresponding to index
def get_letter_from_index(index):
    return chr(EMNIST_df_mapping.loc[index]['lc'])

In [ ]:
# return the index corresponding to letter
def get_index_from_letter(letter):
    return EMNIST_df_mapping[EMNIST_df_mapping['lc']==ord(letter)].index[0]

In [ ]:
# do a scatter with colorful output and that remove filling data (ie: [-10,-10])
def draw_points(X,Y):
    X= np.delete(X, np.where(X == -10))
    Y= np.delete(Y, np.where(Y == -10))

    color_gradient = Color("green").range_to(Color("red"),len(X))
    color_gradient = [color.hex for color in color_gradient]

    plt.plot(X,Y,color="k", label="Linear interpolation")
    plt.scatter(X,Y,c=color_gradient,label="Points")
    plt.title(f'Points: {X.shape[0]}')
    plt.legend()
    plt.axis('equal')
    plt.axis('off')

In [ ]:
# From an image, find the corresponding initial data point from the target dataset and plot then
def plot_original(image):
    flattened_array= image.flatten()
    correct_row = None
    for index, row in target_df.iterrows():
        # Check if the current row is equal to the flattened array
        if np.array_equal(row['images'], flattened_array):
            print(f"Arrays are equal at index {index}")
            correct_row = row
            break

        plt.figure()
        draw_points(correct_row['X'],correct_row['Y'])


In [ ]:
# get the image corresponding to the provided list of X values
def get_image_from_X(X,df):
    for index, row in df.iterrows():
        # Check if the current row is equal to the flattened array
        if np.array_equal(row['X'], X):
            print(f"Arrays are equal at index {index}")
            return row['images'].reshape(image_shape,image_shape)

In [ ]:
# Load the specified try
def load_try(try_name):
    # get the name of the model used for the try
    with open(f'{save_weights_path}/{try_name}/model_type.txt', newline='') as csvfile:
        model_name = csvfile.readline()

    # load the model
    model = load_model(f'{save_weights_path}/initialization/{model_name}.h5')

    # load weights from the try
    model.load_weights(f'{save_weights_path}/{try_name}/data.h5')

    print(f'Loading model {model_name} for try {try_name}')

    return model

In [ ]:
# create a moving average without modifying the value that cannot be averaged
def moving_average(a,smooth_value=5):
    moving_average = np.convolve(a,np.ones(smooth_value)/smooth_value,mode='valid') # do the average
    moving_average = np.append(moving_average,a[-(smooth_value//2):]) # add points at the beginning that can't be smoothed
    moving_average = np.insert(moving_average,0,a[:smooth_value//2]) # add points at the end that can't be smoothed
    return moving_average

# smooth the provided xy coordinates
def smoothen_points(xy_shift, smooth_value):
    x,y = xy_shift.T
    x,y = moving_average(x,smooth_value),moving_average(y,smooth_value)

    return np.array([x,y]).T

# Load Datasets

## Load EMNIST df

In [ ]:
# open the mapping between the index and asci value of characters
dataframe_type  = "emnist-byclass"
EMNIST_df_mapping = pd.read_csv(f'{EMNIST_path}/{dataframe_type}-mapping.txt',header=None,delimiter=" ",index_col=0,names=["lc"])

# get the indices corresponding to the lowercase characters from the alphabet
index_lowercase = list(EMNIST_df_mapping[EMNIST_df_mapping["lc"].isin([97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122])].index)

In [ ]:
if np.isin("EMNIST",source_dataset): # to avoid loading too much into memory
    # read EMNIST_df with figures, lowercase, upercase
    EMNIST_df= pd.read_csv(f'{EMNIST_path}/{dataframe_type}-train.csv',header=None, dtype=np.uint8)

    # keep only the lowercase letters
    EMNIST_df = EMNIST_df[EMNIST_df[0].isin(index_lowercase)]

In [ ]:
# Look at a random sample of the dataset
if np.isin("EMNIST",source_dataset):
    row = EMNIST_df.iloc[np.random.randint(0,EMNIST_df.shape[0])]

    image = np.flip(np.rot90(row.values[1:].reshape((image_shape,image_shape)),axes=(1,0)),axis=1)
    plt.imshow(image,"gray")
    plt.title(get_letter_from_index(row.values[0]))
    plt.show()

## Load digilets dataset

In [ ]:
if np.isin("DigiLeTs",source_dataset): # to avoid loading too much into memory
    digilet_df = pd.read_csv(f'{preprocessed_datasets_path}/digilets-inter3-28-21.csv')
    
    # transform only the necessary features for the type of network to avoid overloading the memory
    if np.isin("CNN",network_type):
        digilet_df['images'] = digilet_df['images'].apply(ast.literal_eval)
        digilet_df['images'] = digilet_df['images'].apply(np.array)
    if np.isin("LSTM",network_type):
        digilet_df['X'] = digilet_df['X'].apply(lambda x: x.replace("nan","-10")) # replace nan with a defined value to be able to eliminate them
        digilet_df['X'] = digilet_df['X'].apply(eval)
        digilet_df['X'] = digilet_df['X'].apply(np.array)
        digilet_df['Y'] = digilet_df['Y'].apply(lambda x: x.replace("nan","-10")) # replace nan with a defined value to be able to eliminate them
        digilet_df['Y'] = digilet_df['Y'].apply(eval)
        digilet_df['Y'] = digilet_df['Y'].apply(np.array)

In [ ]:
# Look at a sample of the data
if np.isin("DigiLeTs",source_dataset) and np.isin("CNN",network_type):
    # look at the data
    row = digilet_df.iloc[np.random.randint(0,digilet_df.shape[0])]

    plt.imshow(row["images"].reshape((image_shape,image_shape)),"gray")
    plt.title(row["letter"])
    plt.show()

## Load BRUSH dataset

In [ ]:
if np.isin("BRUSH",source_dataset): # to avoid loading too much into memory
    brush_df = pd.read_csv(f'{preprocessed_datasets_path}/BRUSH-inter3--.csv')
    
    # transform only the necessary features for the type of network to avoid overloading the memory
    if np.isin("CNN",network_type):
        brush_df['images'] = brush_df['images'].apply(ast.literal_eval)
        brush_df['images'] = brush_df['images'].apply(np.array)
    if np.isin("LSTM",network_type):
        brush_df['X'] = brush_df['X'].apply(lambda x: x.replace("nan","-10")) # replace nan with a defined value to be able to eliminate them
        brush_df['X'] = brush_df['X'].apply(eval)
        brush_df['X'] = brush_df['X'].apply(np.array)
        brush_df['Y'] = brush_df['Y'].apply(lambda x: x.replace("nan","-10")) # replace nan with a defined value to be able to eliminate them
        brush_df['Y'] = brush_df['Y'].apply(eval)
        brush_df['Y'] = brush_df['Y'].apply(np.array)

## Load the target dataset


In [ ]:
target_df = pd.read_csv(f'{preprocessed_datasets_path}/multi_user-inter3-28-21.csv')
interpolate = True

# Apply only the necessary transformation to avoid overloading the memory
if np.isin("CNN",network_type):
    target_df['images'] = target_df['images'].apply(ast.literal_eval)
    target_df['images'] = target_df['images'].apply(np.array)
if np.isin("LSTM",network_type):
    target_df['X'] = target_df['X'].apply(ast.literal_eval)
    target_df['X'] = target_df['X'].apply(np.array)
    target_df['Y'] = target_df['Y'].apply(ast.literal_eval)
    target_df['Y'] = target_df['Y'].apply(np.array)
    # if the data is not interpolated (ie: is not the same length) pad to 32 to have an even length
    if not interpolate:
        target_df['X'] = target_df['X'].apply(lambda x: np.pad(x, (0, 32 - len(x)), constant_values=-10)) 
        target_df['Y'] = target_df['Y'].apply(lambda y: np.pad(y, (0, 32 - len(y)), constant_values=-10))


In [ ]:
# Look at the data
if np.isin("CNN",network_type):
    row = target_df.iloc[np.random.randint(0,target_df.shape[0])]
    image = row['images'].reshape((image_shape,image_shape))
    plt.imshow(image,"gray")
    plt.title(row['letter'])
    plt.show()
    # plot_original(image)

# Prepare the datasets

## Prepare the EMNIST dataset

In [ ]:
# Returns the DigiLeTs dataset in the correct format for the neural network
def get_EMNIST_X_Y(max_number_for_each_letter,random_seed):
    EMNIST_X = ((EMNIST_df.values[:,1:]).reshape(EMNIST_df.shape[0],image_shape,image_shape))
    
    # flip upside down
    EMNIST_X = np.rot90(EMNIST_X, k=1, axes=(2,1))
    EMNIST_X= np.flip(EMNIST_X,axis=2)

    EMNIST_Y= EMNIST_df.values[:,0]

    # A shuffling is required to be done since only the first few examples are taken into account
    EMNIST_X, EMNIST_Y = return_subset(max_number_for_each_letter,EMNIST_X,EMNIST_Y)
    EMNIST_X, EMNIST_Y = shuffle(EMNIST_X, EMNIST_Y,random_state=random_seed)
    return  EMNIST_X, EMNIST_Y

## Prepare Digilets dataset

In [ ]:
# Returns the DigiLeTs dataset in the correct format for the neural network
def get_DigiLeTs_X_Y():
    DigiLeTs_Y = digilet_df['letter'].apply(get_index_from_letter).values # convert letters to numbers
    
    if np.isin("CNN",network_type):
        DigiLeTs_X = np.vstack(digilet_df['images'].values).reshape((digilet_df.shape[0],image_shape,image_shape))
    if np.isin("LSTM",network_type):
        DigiLeTs_X = np.array([np.vstack(digilet_df['X'].values), np.vstack(digilet_df['Y'].values)]).transpose(1,2,0) # reshape the data to fit the LSTM

    return DigiLeTs_X,DigiLeTs_Y

## Prepare BRUSH dataset

In [ ]:
# Returns the BRUSH dataset in the correct format for the neural network
def get_BRUSH_X_Y():
    BRUSH_Y = brush_df['letter'].apply(get_index_from_letter).values # convert letters to numbers
    
    if np.isin("CNN",network_type):
        BRUSH_X = np.vstack(brush_df['images'].values).reshape((brush_df.shape[0],image_shape,image_shape))
    if np.isin("LSTM",network_type):
        BRUSH_X = np.array([np.vstack(brush_df['X'].values), np.vstack(brush_df['Y'].values)]).transpose(1,2,0) # reshape the data to fit the LSTM

    return BRUSH_X,BRUSH_Y

## Prepare target dataset

In [ ]:
# Returns the target dataset in the correct format for the neural network
# If the person to exclude is None, it returns a train, dev(200), test(200) and same(200) distribution split
# If the person to exclude is defined it returns a train, dev(99), test(1) with the dev containing data from the excluded person
def get_target_X_Y(DigiLeTs_shape, person_to_exclude):
    target_df_subset = target_df[target_df['name']!=person_to_exclude]

    target_Y = target_df_subset['letter'].apply(get_index_from_letter).values # convert letters to numbers
    if np.isin("CNN",network_type):
        target_X = np.vstack(target_df_subset['images'].values).reshape((target_df_subset.shape[0],image_shape,image_shape))
    if np.isin("LSTM",network_type):
        target_X = np.array([np.vstack(target_df_subset['X'].values), np.vstack(target_df_subset['Y'].values)]).transpose(1,2,0) # reshape the data to fit the LSTM

        # pad the LSTM value so that it has the same shape that the DigiLeTs values
        target_X = np.pad(target_X, ((0, 0), (0, DigiLeTs_shape-target_X.shape[1]), (0, 0)), constant_values=[-10,-10])

    if np.isin(person_to_exclude,target_df['name']): # test if the person the exclude exist
        excluded_person_df = target_df[target_df['name']==person_to_exclude]
        excluded_Y = excluded_person_df['letter'].apply(get_index_from_letter).values # convert letters to numbers
        if np.isin("CNN",network_type):
            excluded_X = np.vstack(excluded_person_df['images'].values).reshape((excluded_person_df.shape[0],image_shape,image_shape))
        if np.isin("LSTM",network_type):
            excluded_X = np.array([np.vstack(excluded_person_df['X'].values), np.vstack(excluded_person_df['Y'].values)]).transpose(1,2,0) # reshape the data to fit the LSTM
            excluded_X = np.pad(excluded_X, ((0, 0), (0, DigiLeTs_shape-excluded_X.shape[1]), (0, 0)), constant_values=[-10,-10])
    else:
        excluded_X, excluded_Y = None, None

    return (target_X,target_Y,excluded_X,excluded_Y)

# Split in train, dev and test

In [ ]:
def split_train_dev_test_dataset(test_dev_df_size,EMNIST_data_multiplier,random_seed,person_to_exclude=None):
    person_independent_mode = np.isin(person_to_exclude,target_df['name'])
    if person_to_exclude is not None and not person_independent_mode:
        raise ValueError("Wrong name for person independent")

    # Initialise every variables
    train_X, test_same_dist_X, train_Y_categorical, test_same_dist_Y_categorical,source_X_train, source_X_test, target_X_train, source_Y_train_categorical, source_Y_test_categorical, target_Y_train_categorical, target_Y_train,source_Y_test,source_Y_train,train_Y,test_same_dist_Y = None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
    
    # take the correct number of examples from the EMNIST. Use one sample per letter (26 letter) when not EMNIST data is required to avoid breaking the slip 
    number_examples_from_source =26 if EMNIST_data_multiplier==0 else (target_df.shape[0] - 2*test_dev_df_size)*EMNIST_data_multiplier

    # set the correct dataset and the correct mode
    if np.isin("BRUSH",source_dataset) and np.isin("DigiLeTs",source_dataset):
        source_X_D,source_Y_D = get_DigiLeTs_X_Y()
        source_X_B,source_Y_B = get_BRUSH_X_Y()
        source_X,source_Y = np.concatenate((source_X_D,source_X_B)),np.concatenate((source_Y_D,source_Y_B))
    elif np.isin("EMNIST",source_dataset) and np.isin("DigiLeTs",source_dataset):
        source_X_D,source_Y_D = get_DigiLeTs_X_Y()
        source_X_E,source_Y_E = get_EMNIST_X_Y(max_number_for_each_letter=2000,random_seed = random_seed)
        # add in priority the digilets dataset and then the EMNIST dataset and keep only what is going to be kept at the end
        source_X,source_Y = np.concatenate((source_X_D,source_X_E))[:number_examples_from_source+26],np.concatenate((source_Y_D,source_Y_E))[:number_examples_from_source+26]
    elif np.isin("DigiLeTs",source_dataset):
        source_X,source_Y = get_DigiLeTs_X_Y()
    elif np.isin("EMNIST",source_dataset):
        source_X,source_Y = get_EMNIST_X_Y(max_number_for_each_letter=2000,random_seed =random_seed)

    # Shuffling required because sometimes datasets are just concatenated
    source_X, source_Y = shuffle(source_X, source_Y,random_state=random_seed)

    target_X, target_Y, excluded_X, excluded_Y = get_target_X_Y(DigiLeTs_shape= source_X.shape[1], person_to_exclude = person_to_exclude)

    # create a relatively small dev and test set (to be able to detect 0.5% improvement) to add as many example as possible in the train set
    # stratify the datasets to be able to have a correct representation of all the class in the test/dev set
    if not person_independent_mode:
        target_X_remaining, test_X, target_Y_remaining, test_Y = train_test_split(target_X,target_Y,random_state=random_seed,stratify=target_Y,test_size=test_dev_df_size)
        target_X_remaining, dev_X, target_Y_remaining, dev_Y = train_test_split(target_X_remaining,target_Y_remaining,random_state=random_seed,stratify=target_Y_remaining,test_size=test_dev_df_size)
    else:
        target_X_remaining, target_Y_remaining = target_X, target_Y
        dev_X, dev_Y = shuffle(excluded_X[:-1], excluded_Y[:-1],random_state=random_seed)
        test_X, test_Y = shuffle(np.expand_dims(excluded_X[-1], axis=0), np.expand_dims(excluded_Y[-1], axis=0),random_state=random_seed)

    # get a fraction of the source data that is stratified
    _ , source_X_fraction, _, source_Y_fraction =  train_test_split(source_X,source_Y,random_state=random_seed,test_size=number_examples_from_source, stratify=source_Y)

    # shift labels so that they are between 0 and 25
    test_Y_categorical = to_categorical(test_Y-36, num_classes=26)
    dev_Y_categorical = to_categorical(dev_Y-36, num_classes=26)

    # for normal training
    if training_mode=="Normal":
        train_X = np.concatenate((source_X_fraction,target_X_remaining))
        train_Y = np.concatenate((source_Y_fraction,target_Y_remaining))
        if not person_independent_mode:
            train_X, test_same_dist_X ,train_Y, test_same_dist_Y = train_test_split(train_X,train_Y,random_state=random_seed,test_size=test_dev_df_size,stratify=train_Y)
            test_same_dist_Y_categorical = to_categorical(test_same_dist_Y-36, num_classes=26)
        else:
            train_X,train_Y = shuffle(train_X,train_Y,random_state=random_seed)

        train_Y_categorical = to_categorical(train_Y-36, num_classes=26)

    # for fine tuning
    elif training_mode=="Fine-tuned":
        source_X_train, source_X_test, source_Y_train, source_Y_test = train_test_split(source_X_fraction,source_Y_fraction,stratify=source_Y_fraction, test_size=test_dev_df_size,random_state=random_seed)
        target_X_train, target_Y_train = target_X_remaining, target_Y_remaining

        source_Y_train_categorical = to_categorical(source_Y_train-36, num_classes=26)
        source_Y_test_categorical = to_categorical(source_Y_test-36, num_classes=26)
        target_Y_train_categorical = to_categorical(target_Y_train-36, num_classes=26)

    # pack the data to be returned
    data_normal_training = (train_X, test_same_dist_X, dev_X, test_X, train_Y_categorical, test_same_dist_Y_categorical, dev_Y_categorical, test_Y_categorical)
    data_fine_tuning = (source_X_train, source_X_test, target_X_train, dev_X, test_X, source_Y_train_categorical, source_Y_test_categorical, target_Y_train_categorical, dev_Y_categorical, test_Y_categorical)
    accessory = (train_Y,test_same_dist_Y,source_Y_train,source_Y_test,target_Y_train,dev_Y,test_Y)
    return (data_normal_training,data_fine_tuning,accessory)

In [ ]:
# example of querying the data
data_normal_training,data_fine_tuning,accessory = split_train_dev_test_dataset(test_dev_df_size=200,EMNIST_data_multiplier=8,random_seed=42, person_to_exclude=None)

# unpack
train_X, test_same_dist_X, dev_X, test_X, train_Y_categorical, test_same_dist_Y_categorical, dev_Y_categorical, test_Y_categorical = data_normal_training
source_X_train, source_X_test, target_X_train, dev_X, test_X, source_Y_train_categorical, source_Y_test_categorical, target_Y_train_categorical, dev_Y_categorical, test_Y_categorical = data_fine_tuning
train_Y,test_same_dist_Y,source_Y_train,source_Y_test,target_Y_train, dev_Y,test_Y = accessory

# Create the model

## Model 1

In [ ]:
if np.isin("CNN",network_type):
    model1 = Sequential()
    model1.add(Conv2D(32, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))
    model1.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))
    model1.add(Flatten())
    model1.add(Dense(128, activation='relu'))
    model1.add(Dense(26, activation='softmax'))
    model1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model1.summary()

In [ ]:
# model 1 for different shapes
if np.isin("CNN",network_type):
    model1_1 = Sequential()
    model1_1.add(Conv2D(32, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model1_1.add(MaxPooling2D(pool_size=(2, 2)))
    model1_1.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model1_1.add(MaxPooling2D(pool_size=(2, 2)))
    model1_1.add(Flatten())
    model1_1.add(Dense(128, activation='relu'))
    model1_1.add(Dense(26, activation='softmax'))
    model1_1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model1_1.summary()

In [ ]:
# Model 1 but with old version of tensorflow to be able to use the GPU with direct ML
if np.isin("CNN",network_type):
    model1_2 = Sequential()
    model1_2.add(Conv2D(32, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model1_2.add(MaxPooling2D(pool_size=(2, 2)))
    model1_2.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model1_2.add(MaxPooling2D(pool_size=(2, 2)))
    model1_2.add(Flatten())
    model1_2.add(Dense(128, activation='relu'))
    model1_2.add(Dense(26, activation='softmax'))
    model1_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model1_2.summary()

## Model 2

In [ ]:
if np.isin("CNN",network_type):
    model2 = Sequential()
    model2.add(Conv2D(32, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model2.add(BatchNormalization())
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model2.add(BatchNormalization())
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Flatten())
    model2.add(Dense(128, activation='relu'))
    model2.add(BatchNormalization())
    model2.add(Dense(26, activation='softmax'))
    model2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model2.summary()

## Model 3

In [ ]:
if np.isin("CNN",network_type):
    model3 = Sequential()
    model3.add(Conv2D(32, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(MaxPooling2D(pool_size=(2, 2)))
    model3.add(Dropout(0.5))
    model3.add(Flatten())
    model3.add(Dense(128, activation='relu'))
    model3.add(BatchNormalization())
    model3.add(Dropout(0.5))
    model3.add(Dense(26, activation='softmax'))
    model3.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model3.summary()

## Model 4

In [ ]:
# simpler model 1
if np.isin("CNN",network_type):
    model4 = Sequential()
    model4.add(Conv2D(32, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model4.add(BatchNormalization())
    model4.add(MaxPooling2D(pool_size=(2, 2)))
    model4.add(Dropout(0.2))
    model4.add(Flatten())
    model4.add(Dense(128, activation='relu'))
    model4.add(BatchNormalization())
    model4.add(Dropout(0.2))
    model4.add(Dense(26, activation='softmax'))
    model4.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model4.summary()

## Model 5

In [ ]:
if np.isin("CNN",network_type):
    model5 = Sequential()
    model5.add(Conv2D(32, kernel_size=(5, 5), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model5.add(MaxPooling2D(pool_size=(2, 2)))
    model5.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
    model5.add(MaxPooling2D(pool_size=(2, 2)))
    model5.add(Flatten())
    model5.add(Dense(128, activation='relu'))
    model5.add(Dense(26, activation='softmax'))
    model5.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model5.summary()

## Model 6

In [ ]:
if np.isin("CNN",network_type):
    model6 = Sequential()
    model6.add(Conv2D(16, kernel_size=(5, 5), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model6.add(MaxPooling2D(pool_size=(2, 2)))
    model6.add(Flatten())
    model6.add(Dense(64, activation='relu'))
    model6.add(Dense(26, activation='softmax'))
    model6.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model6.summary()

## Model 7

In [ ]:
if np.isin("CNN",network_type):
    model7 = Sequential()
    model7.add(Conv2D(8, kernel_size=(5, 5), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model7.add(MaxPooling2D(pool_size=(2, 2)))
    model7.add(Flatten())
    model7.add(Dense(26, activation='softmax'))
    model7.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model7.summary()

## Model 8

In [ ]:
if np.isin("CNN",network_type):
    model8 = Sequential()
    model8.add(Conv2D(4, kernel_size=(5, 5), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model8.add(MaxPooling2D(pool_size=(2, 2)))
    model8.add(Flatten())
    model8.add(Dense(26, activation='softmax'))
    model8.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model8.summary()

## Model 9

In [ ]:
if np.isin("CNN",network_type):
    model9 = Sequential()
    model9.add(Conv2D(4, kernel_size=(5, 5), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model9.add(MaxPooling2D(pool_size=(2, 2)))
    model9.add(Dropout(0.25))  # Adjust the dropout rate as needed
    model9.add(Flatten())
    model9.add(Dense(26, activation='softmax'))
    model9.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model9.summary()

## Model 10

In [ ]:
# try to add a second convolutional layer to have a chance of capturing info in several levels
if np.isin("CNN",network_type):
    model10 = Sequential()
    model10.add(Conv2D(4, kernel_size=(5, 5), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model10.add(MaxPooling2D(pool_size=(2, 2)))
    model10.add(Dropout(0.25))  # Adjust the dropout rate as needed
    model10.add(Conv2D(4, kernel_size=(5, 5), activation='relu'))
    model10.add(MaxPooling2D(pool_size=(2, 2)))
    model10.add(Dropout(0.25))  # Adjust the dropout rate as needed
    model10.add(Flatten())
    model10.add(Dense(26, activation='softmax'))
    model10.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model10.summary()

## Model 11

In [ ]:
if np.isin("CNN",network_type):
    model11 = Sequential()
    model11.add(Conv2D(4, kernel_size=(5, 5), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model11.add(MaxPooling2D(pool_size=(2, 2)))
    model11.add(Conv2D(4, kernel_size=(5, 5), activation='relu'))
    model11.add(MaxPooling2D(pool_size=(2, 2)))
    model11.add(Flatten())
    model11.add(Dense(26, activation='softmax'))
    model11.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model11.summary()

## Model test

In [ ]:
if np.isin("CNN",network_type):
    model12 = Sequential()
    model12.add(Conv2D(16, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model12.add(MaxPooling2D(pool_size=(2, 2)))
    model12.add(Dropout(0.20))
    model12.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model12.add(MaxPooling2D(pool_size=(2, 2)))
    model12.add(Flatten())
    model12.add(Dense(26, activation='softmax'))
    model12.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model12.summary()

## Model 13

In [ ]:
if np.isin("CNN",network_type):
    model13 = Sequential()
    model13.add(Conv2D(16, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model13.add(MaxPooling2D(pool_size=(2, 2)))
    model13.add(Dropout(0.20))
    model13.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model13.add(MaxPooling2D(pool_size=(2, 2)))
    model13.add(Flatten())
    model13.add(Dense(26, activation='softmax'))
    model13.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model13.summary()

## Model 14

In [ ]:
if np.isin("CNN",network_type):
    model14 = Sequential()
    model14.add(Conv2D(16, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model14.add(MaxPooling2D(pool_size=(2, 2)))
    model14.add(Dropout(0.20))
    model14.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model14.add(MaxPooling2D(pool_size=(2, 2)))
    model14.add(Flatten())
    model14.add(Dense(128, activation='relu'))
    model14.add(Dense(26, activation='softmax'))
    model14.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model14.summary()

## Model 15 - LSTM

In [ ]:
# same LSTM model that had great results during the protech
if np.isin("LSTM",network_type):
    model15 = Sequential()
    model15.add(LSTM(50, return_sequences=True, input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model15.add(Dropout(0.2))
    model15.add(LSTM(50, return_sequences=False))
    model15.add(Dropout(0.2))
    model15.add(Dense(26, activation='softmax'))

    model15.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model15.summary()

## Model 16 - LSTM

In [ ]:
if np.isin("LSTM",network_type):
    model16 = Sequential()
    model16.add(Masking(mask_value = [-10,-10], input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model16.add(LSTM(50))
    model16.add(Dense(26, activation='softmax'))

    model16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model16.summary()

## Model 17 - LSTM

In [ ]:
if np.isin("LSTM",network_type):
    model17 = Sequential()
    model17.add(Masking(mask_value = [-10,-10], input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model17.add(Bidirectional(LSTM(50)))
    model17.add(Dense(26, activation='softmax'))
    model17.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model17.summary()

## Model 18

In [ ]:
# different shape of 16
if np.isin("LSTM",network_type):
    model18 = Sequential()
    model18.add(Masking(mask_value = [-10,-10], input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model18.add(LSTM(50))
    model18.add(Dense(26, activation='softmax'))
    model18.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model18.summary()

## Model 19

In [ ]:
# same 17 but with different shapes
if np.isin("LSTM",network_type):
    model19 = Sequential()
    model19.add(Masking(mask_value = [-10,-10], input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model19.add(Bidirectional(LSTM(50)))
    model19.add(Dense(26, activation='softmax'))
    model19.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model19.summary()

## Model 20

In [ ]:
if np.isin("CNN",network_type):
    model20 = Sequential()
    model20.add(Conv2D(32, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model20.add(MaxPooling2D(pool_size=(2, 2)))
    model20.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model20.add(MaxPooling2D(pool_size=(2, 2)))
    model20.add(Flatten())
    model20.add(Dense(128, activation='relu'))
    model20.add(Dropout(0.25))
    model20.add(Dense(26, activation='softmax'))
    model20.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model20.summary()

## Model 21

In [ ]:
if np.isin("CNN",network_type):
    model21 = Sequential()
    model21.add(Conv2D(32, strides=3, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model21.add(Conv2D(64, strides=3, kernel_size=(3, 3), activation='relu'))
    model21.add(Flatten())
    model21.add(Dense(128, activation='relu'))
    model21.add(Dropout(0.25))
    model21.add(Dense(26, activation='softmax'))
    model21.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model21.summary()

## Model 22

In [ ]:
if np.isin("CNN",network_type):
    model22 = Sequential()
    model22.add(Conv2D(32, strides=2, kernel_size=(3, 3), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model22.add(Conv2D(64, strides=2, kernel_size=(3, 3), activation='relu'))
    model22.add(Flatten())
    model22.add(Dense(128, activation='relu'))
    model22.add(Dropout(0.25))
    model22.add(Dense(26, activation='softmax'))
    model22.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model22.summary()

## Model 23

In [ ]:
if np.isin("CNN",network_type):
    model23 = Sequential()
    model23.add(Conv2D(32, strides=3, kernel_size=(7, 7), input_shape=(image_shape, image_shape, 1), activation='relu'))
    model23.add(Conv2D(64, strides=3, kernel_size=(7,7), activation='relu'))
    model23.add(Flatten())
    model23.add(Dense(128, activation='relu'))
    model23.add(Dropout(0.25))
    model23.add(Dense(26, activation='softmax'))
    model23.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model23.summary()

## Model 24

In [ ]:
# Same model than what was used in the protech
if np.isin("LSTM",network_type):
    model24 = Sequential()
    model24.add(LSTM(50, return_sequences=True, input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model24.add(Dropout(0.2))
    model24.add(LSTM(50, return_sequences=False))
    model24.add(Dropout(0.2))
    model24.add(Dense(26, activation='softmax'))
    model24.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model24.summary()

## Model 25

In [ ]:
# same 19 but with a dropout
if np.isin("LSTM",network_type):
    model25 = Sequential()
    model25.add(Masking(mask_value = [-10,-10], input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model25.add(Bidirectional(LSTM(50)))
    model25.add(Dropout(0.2)) # to improve generalization
    model25.add(Dense(26, activation='softmax'))
    model25.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model25.summary()

## Model 26

In [ ]:
# same 19 but with more units
if np.isin("LSTM",network_type):
    model26 = Sequential()
    model26.add(Masking(mask_value = [-10,-10], input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model26.add(Bidirectional(LSTM(100)))
    model26.add(Dense(26, activation='softmax'))
    model26.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model26.summary()

## Model 27

In [ ]:
# same 19 but with less units
if np.isin("LSTM",network_type):
    model27 = Sequential()
    model27.add(Masking(mask_value = [-10,-10], input_shape=(dev_X.shape[1], dev_X.shape[2])))
    model27.add(Bidirectional(LSTM(25)))
    model27.add(Dense(26, activation='softmax'))
    model27.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model27.summary()

# Train

## Train model in one go

In [ ]:
if training_mode=="Normal" and not directML:
    max_epochs = 200
    model_name = "model1"
    try_name = '52-CNN'
    patience = 30

    folder_to_save = f'{save_weights_path}/{try_name}'

    history = AdditionalValidationSets([(test_same_dist_X, test_same_dist_Y_categorical, 'test_same_dist')])

    # if the model doesn't improve on the validation data accuracy for 3 epochs, stop the training
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=patience, restore_best_weights=True, mode='max')

    # Save model with best validation accuracy
    checkpoint = ModelCheckpoint(
        folder_to_save+'/data.h5',  # Specify the file to save the weights
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
        verbose=1
    )

    csv_logger = CSVLogger(folder_to_save + "/training_history.csv")  # Save the history has soon as it is computed

    # load the model if it already exist and save the new one if it doesn't already exist
    # this allows to always have the same weights and make sure they are reset for a new training
    model_initialization_path = f'{save_weights_path}/initialization/{model_name}.h5'
    if model_name == "modelTest": # do not save the model when quickly iterating
        model = eval(model_name)
    elif not os.path.isfile(model_initialization_path):
        model = eval(model_name) # load the model declared above
        save_model(model,model_initialization_path)
        print("Saving "+model_name)
    else:
        if(input("This MODEL already exist, do you want to overwrite it ? (y/n)")=='y'):
            model = eval(model_name) # load the model declared above
            save_model(model,model_initialization_path)
            print("Saving "+model_name)
        else:
            model = load_model(model_initialization_path)
            print("Loading ",model_name)

    overwrite = True
    # create the folder saving this try  if it doesn't already exist
    if not os.path.isdir(folder_to_save):
        os.mkdir(folder_to_save) # create the folder
    else:
        overwrite = input("This folder already exist, do you want to overwrite it ? (y/n)")=='y'

    if overwrite:
        with open(f'{folder_to_save}/model_type.txt', 'w') as f:
            f.write(model_name)

        model.fit(
            train_X, train_Y_categorical,
            epochs=max_epochs,
            batch_size=32,
            validation_data=(dev_X, dev_Y_categorical),
            callbacks=[checkpoint, csv_logger, early_stopping,history]
        )
        history = history.history #keep only the useful part
        pd.DataFrame(history).to_csv(folder_to_save + "/training_history.csv") # save the history containing the 2nd validation dataset

In [ ]:
# Same than the old but with old version of tensorflow to be able to use the GPU with direct ML
if training_mode=="Normal" and directML:
    max_epochs = 200
    model_name = "model23"
    try_name = '56-CNN'
    patience = 30

    folder_to_save = f'{save_weights_path}/{try_name}'

    a,b,c = train_X.shape
    history = AdditionalValidationSets([(test_same_dist_X.reshape(test_same_dist_X.shape[0],b,c,1), test_same_dist_Y_categorical, 'test_same_dist')])

    # if the model doesn't improve on the validation data accuracy for 3 epochs, stop the training
    early_stopping = EarlyStopping(monitor='val_acc', patience=patience, restore_best_weights=True, mode='max')

    # Save model with best validation accuracy
    checkpoint = ModelCheckpoint(
        folder_to_save+'/data.h5',  # Specify the file to save the weights
        monitor='val_acc',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
        verbose=1
    )

    csv_logger = CSVLogger(folder_to_save + "/training_history.csv")  # Save the history has soon as it is computed

    # load the model if it already exist and save the new one if it doesn't already exist
    # this allows to always have the same weights and make sure they are reset for a new training
    model_initialization_path = f'{save_weights_path}/initialization/{model_name}.h5'
    if model_name == "modelTest": # do not save the model when quickly iterating
        model = eval(model_name)
    elif not os.path.isfile(model_initialization_path):
        model = eval(model_name) # load the model declared above
        save_model(model,model_initialization_path)
        print("Saving "+model_name)
    else:
        if(input("This MODEL already exist, do you want to overwrite it ? (y/n)")=='y'):
            model = eval(model_name) # load the model declared above
            save_model(model,model_initialization_path)
            print("Saving "+model_name)
        else:
            model = load_model(model_initialization_path)
            print("Loading ",model_name)

    overwrite = True
    # create the folder saving this try  if it doesn't already exist
    if not os.path.isdir(folder_to_save):
        os.mkdir(folder_to_save) # create the folder
    else:
        overwrite = input("This folder already exist, do you want to overwrite it ? (y/n)")=='y'

    if overwrite:
        with open(f'{folder_to_save}/model_type.txt', 'w') as f:
            f.write(model_name)

        try:
            model.fit(
                train_X.reshape(a,b,c,1), train_Y_categorical,
                epochs=max_epochs,
                batch_size=32,
                validation_data=(dev_X.reshape(dev_X.shape[0],b,c,1), dev_Y_categorical),
                callbacks=[checkpoint, csv_logger, early_stopping,history]
            )
        finally:
            history = history.history #keep only the useful part
            pd.DataFrame(history).rename(columns={'acc':'accuracy','val_acc':'val_accuracy','test_same_dist_acc':'test_same_dist_accuracy'}).to_csv(folder_to_save + "/training_history.csv").to_csv(folder_to_save + "/training_history.csv") # save the history containing the 2nd validation dataset

## Train with generic data and fine-tune

### Train on source data

In [ ]:
# Load desired dataset
data_normal_training,data_fine_tuning,accessory = split_train_dev_test_dataset(test_dev_df_size=200,EMNIST_data_multiplier=8,random_seed=42, person_to_exclude="David")
source_X_train, source_X_test, target_X_train, dev_X, test_X, source_Y_train_categorical, source_Y_test_categorical, target_Y_train_categorical, dev_Y_categorical, test_Y_categorical = data_fine_tuning
train_Y,test_same_dist_Y,source_Y_train,source_Y_test,target_Y_train, dev_Y,test_Y = accessory

In [ ]:
if training_mode=="Fine-tuned" and not directML:
    max_epochs = 500
    model_name = "model27"
    try_name = 'model27-LSTM'
    patience = 30

    folder_to_save = f'{save_weights_path}/{try_name}'

    # if the model doesn't improve on the validation data accuracy for 3 epochs, stop the training
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=patience, restore_best_weights=True, mode='max')

    # Save model with best validation accuracy
    checkpoint = ModelCheckpoint(
        folder_to_save+'/data.h5',  # Specify the file to save the weights
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
        verbose=1
    )

    csv_logger = CSVLogger(folder_to_save + "/training_history.csv")  # Save the history has soon as it is computed

    # load the model if it already exist and save the new one if it doesn't already exist
    # this allows to always have the same weights and make sure they are reset for a new training
    model_initialization_path = f'{save_weights_path}/initialization/{model_name}.h5'
    if not os.path.isfile(model_initialization_path):
        model = eval(model_name) # load the model declared above
        save_model(model,model_initialization_path)
        print("Saving "+model_name)
    else:
        model = load_model(model_initialization_path)
        # model = load_try("model19-LSTM-BRUSH") # uncomment to be able to continue training the model
        print("Loading ",model_name)

    overwrite = True
    # create the folder saving this try  if it doesn't already exist
    if not os.path.isdir(folder_to_save):
        os.mkdir(folder_to_save) # create the folder
    else:
        overwrite = input("This folder already exist, do you want to overwrite it ? (y/n)")=='y'

    if overwrite:
        with open(f'{folder_to_save}/model_type.txt', 'w') as f:
            f.write(model_name)

        history = model.fit(
            source_X_train, source_Y_train_categorical,
            epochs=max_epochs,
            batch_size=32,
            validation_data=(source_X_test, source_Y_test_categorical),
            callbacks=[checkpoint,csv_logger, early_stopping]
        )

        history = history.history

        pd.DataFrame(history).to_csv(folder_to_save + "/training_history.csv") # save the history containing the 2nd validation dataset

In [ ]:
# Same than the old but with old version of tensorflow to be able to use the GPU with direct ML
if training_mode=="Fine-tuned" and directML:
    max_epochs = 500
    model_name = "model1_2"
    try_name = 'model1_2-CNN-Digi+EMINST-50'
    patience = 30

    a,b,c = source_X_train.shape
    d,_,_ = source_X_test.shape

    folder_to_save = f'{save_weights_path}/{try_name}'

    # if the model doesn't improve on the validation data accuracy for 3 epochs, stop the training
    early_stopping = EarlyStopping(monitor='val_acc', patience=patience, restore_best_weights=True, mode='max')

    # Save model with best validation accuracy
    checkpoint = ModelCheckpoint(
        folder_to_save+'/data.h5',  # Specify the file to save the weights
        monitor='val_acc',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
        verbose=1
    )

    csv_logger = CSVLogger(folder_to_save + "/training_history.csv")  # Save the history has soon as it is computed

    # load the model if it already exist and save the new one if it doesn't already exist
    # this allows to always have the same weights and make sure they are reset for a new training
    model_initialization_path = f'{save_weights_path}/initialization/{model_name}.h5'
    if not os.path.isfile(model_initialization_path):
        model = eval(model_name) # load the model declared above
        save_model(model,model_initialization_path)
        print("Saving "+model_name)
    else:
        model = load_model(model_initialization_path)
        print("Loading ",model_name)

    overwrite = True
    # create the folder saving this try  if it doesn't already exist
    if not os.path.isdir(folder_to_save):
        os.mkdir(folder_to_save) # create the folder
    else:
        overwrite = input("This folder already exist, do you want to overwrite it ? (y/n)")=='y'

    if overwrite:
        with open(f'{folder_to_save}/model_type.txt', 'w') as f:
            f.write(model_name)

        history = model.fit(
            source_X_train.reshape(a,b,c,1), source_Y_train_categorical,
            epochs=max_epochs,
            batch_size=32,
            validation_data=(source_X_test.reshape(d,b,c,1), source_Y_test_categorical),
            callbacks=[checkpoint,csv_logger, early_stopping]
        )

        history = history.history

        pd.DataFrame(history).rename(columns={'acc':'accuracy','val_acc':'val_accuracy','test_same_dist_acc':'test_same_dist_accuracy'}).rename(columns={'acc':'accuracy','val_acc':'val_accuracy','test_same_dist_acc':'test_same_dist_accuracy'}).to_csv(folder_to_save + "/training_history.csv") # save the history containing the 2nd validation dataset

### Fine-tune on target data

In [ ]:
# Fine-tune on CPU
if training_mode=="Fine-tuned" and not directML:
    max_epochs = 200
    batch_size = 64
    model_to_load = "model1_2-CNN-Digi+EMINST"
    try_name = '68-CNN-FT'
    patience = 30
    learning_rate = 0.01 # default 0.001
    number_try = 1
    layer_to_freeze = [] #index of the layer to freeze
    people_to_exclude = []

    initial_model = model_to_load.split('-')[0]
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=patience, restore_best_weights=True, mode='max')
    
    wrong_name = np.any([not np.isin(name,np.unique(target_df['name'])) for name in people_to_exclude])
    # if wrong_name:
    #     print([not np.isin(name,np.unique(target_df['name'])) for name in people_to_exclude])
    #     raise ValueError("Wrong person name")
    # if len(people_to_exclude)==0:
    #     people_to_exclude = [None]

    for person_to_exclude in people_to_exclude:
        if person_to_exclude is not None:
            custom_try_name = f'{try_name}-{person_to_exclude}'
        else:
            custom_try_name = try_name

        # get the right dataset
        data_normal_training,data_fine_tuning,accessory = split_train_dev_test_dataset(test_dev_df_size=200,EMNIST_data_multiplier=8,random_seed=42, person_to_exclude=person_to_exclude)
        source_X_train, source_X_test, target_X_train, dev_X, test_X, source_Y_train_categorical, source_Y_test_categorical, target_Y_train_categorical, dev_Y_categorical, test_Y_categorical = data_fine_tuning

        for try_number in range(number_try):
            if number_try>1:
                folder_to_save = f'{save_weights_path}/{custom_try_name}-{try_number}'
            else:
                folder_to_save = f'{save_weights_path}/{custom_try_name}'

            # if the model doesn't improve on the validation data accuracy for 3 epochs, stop the training

            # Save model with best validation accuracy
            checkpoint = ModelCheckpoint(
                folder_to_save+'/data.h5',  # Specify the file to save the weights
                monitor='val_accuracy',
                save_best_only=True,
                save_weights_only=True,
                mode='max',
                verbose=1
            )

            csv_logger = CSVLogger(folder_to_save + "/training_history.csv")  # Save the history has soon as it is computed

            # load the model if it already exist and save the new one if it doesn't already exist
            # this allows to always have the same weights and make sure they are reset for a new training
            model_initialization_path = f'{save_weights_path}/{model_to_load}/data.h5'
            if not os.path.isfile(model_initialization_path):
                print("The model you are trying to load doesn't exist")
            else:
                model = load_try(model_to_load)
                print("Loading ",model_to_load)

            if len(layer_to_freeze)!=0:
                # freeze layers
                for i,layer in enumerate(model.layers):
                    if np.isin(i,layer_to_freeze):
                        layer.trainable = False
                        print(layer," is no longer trainable")

                # recompile so that the changes take into effect
                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

            # set the learning rate of the model
            K.set_value(model.optimizer.learning_rate, learning_rate)

            overwrite = True
            # create the folder saving this try  if it doesn't already exist
            if not os.path.isdir(folder_to_save):
                os.mkdir(folder_to_save) # create the folder
            else:
                overwrite = input("This folder already exist, do you want to overwrite it ? (y/n)")=='y'

            if overwrite:
                with open(f'{folder_to_save}/model_type.txt', 'w') as f:
                    f.write(initial_model)

                history = model.fit(
                    target_X_train, target_Y_train_categorical,
                    epochs=max_epochs,
                    batch_size=batch_size,
                    validation_data=(dev_X, dev_Y_categorical),
                    callbacks=[checkpoint, csv_logger, early_stopping]
                )

                history = history.history #keep only the useful part
                pd.DataFrame(history).to_csv(folder_to_save + "/training_history.csv") # save the history containing the 2nd validation dataset
            else:
                break

In [ ]:
# Same than the old but with old version of tensorflow to be able to use the GPU with direct ML
if training_mode=="Fine-tuned" and directML:
    max_epochs = 300
    batch_size = 32
    model_to_load = "model1_2-CNN-Digi+EMINST-50"
    try_name = '69-CNN-FT'
    patience = 30
    learning_rate = 0.001 # default 0.001
    number_try = 1
    layer_to_freeze = [] #index of the layer to freeze
    people_to_exclude = []

    initial_model = model_to_load.split('-')[0]
    early_stopping = EarlyStopping(monitor='val_acc', patience=patience, restore_best_weights=True, mode='max')
    
    wrong_name = np.any([not np.isin(name,np.unique(target_df['name'])) for name in people_to_exclude])
    # if wrong_name:
    #     print([not np.isin(name,np.unique(target_df['name'])) for name in people_to_exclude])
    #     raise ValueError("Wrong person name")
    # if len(people_to_exclude)==0:
    #     people_to_exclude = [None]

    for person_to_exclude in people_to_exclude:
        if person_to_exclude is not None:
            custom_try_name = f'{try_name}-{person_to_exclude}'
        else:
            custom_try_name = try_name

        # get the right dataset
        data_normal_training,data_fine_tuning,accessory = split_train_dev_test_dataset(test_dev_df_size=200,EMNIST_data_multiplier=8,random_seed=42, person_to_exclude=person_to_exclude)
        source_X_train, source_X_test, target_X_train, dev_X, test_X, source_Y_train_categorical, source_Y_test_categorical, target_Y_train_categorical, dev_Y_categorical, test_Y_categorical = data_fine_tuning

        for try_number in range(number_try):
            if number_try>1:
                folder_to_save = f'{save_weights_path}/{custom_try_name}-{try_number}'
            else:
                folder_to_save = f'{save_weights_path}/{custom_try_name}'

            # if the model doesn't improve on the validation data accuracy for 3 epochs, stop the training

            # Save model with best validation accuracy
            checkpoint = ModelCheckpoint(
                folder_to_save+'/data.h5',  # Specify the file to save the weights
                monitor='val_acc',
                save_best_only=True,
                save_weights_only=True,
                mode='max',
                verbose=1
            )

            csv_logger = CSVLogger(folder_to_save + "/training_history.csv")  # Save the history has soon as it is computed

            # load the model if it already exist and save the new one if it doesn't already exist
            # this allows to always have the same weights and make sure they are reset for a new training
            model_initialization_path = f'{save_weights_path}/{model_to_load}/data.h5'
            if not os.path.isfile(model_initialization_path):
                raise ValueError("The model you are trying to load doesn't exist")
            else:
                model = load_try(model_to_load)
                print("Loading ",model_to_load)

            if len(layer_to_freeze)!=0:
                # freeze layers
                for i,layer in enumerate(model.layers):
                    if np.isin(i,layer_to_freeze):
                        layer.trainable = False
                        print(layer," is no longer trainable")

                # recompile so that the changes take into effect
                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

            # set the learning rate of the model
            K.set_value(model.optimizer.learning_rate, learning_rate)

            overwrite = True
            # create the folder saving this try  if it doesn't already exist
            if not os.path.isdir(folder_to_save):
                os.mkdir(folder_to_save) # create the folder
            else:
                overwrite = input("This folder already exist, do you want to overwrite it ? (y/n)")=='y'

            if overwrite:
                with open(f'{folder_to_save}/model_type.txt', 'w') as f:
                    f.write(initial_model)

                a,b,c = target_X_train.shape
                d,_,_ = dev_X.shape

                history = model.fit(
                    target_X_train.reshape(a,b,c,1), target_Y_train_categorical,
                    epochs=max_epochs,
                    batch_size=batch_size,
                    validation_data=(dev_X.reshape(d,b,c,1), dev_Y_categorical),
                    callbacks=[checkpoint, csv_logger, early_stopping]
                )

                history = history.history #keep only the useful part
                pd.DataFrame(history).rename(columns={'acc':'accuracy','val_acc':'val_accuracy','test_same_dist_acc':'test_same_dist_accuracy'}).to_csv(folder_to_save + "/training_history.csv") # save the history containing the 2nd validation dataset
            else:
                break

In [ ]:
raise SystemExit("Stop right there!") # stop the run when running everything in one go

# Visualize result

## Load history from CSV

In [ ]:
# Load the results by using the name of the model or a regex expression
models_to_comparee = ['68-CNN-FT-[0-9]**']
plot_training = True
plot_dev = True
plot_same_div = True if training_mode == "Normal" else False
smoothened_plot = True
smoothness_level = 10
max_search_beginning = 0

path_to_compare = np.array([])
for models_to_compare in models_to_comparee:
    path_to_compare = np.concatenate((path_to_compare,np.array(glob.glob(f'{save_weights_path}/{models_to_compare}'))))


histories = [(pd.read_csv(f'{model_path}/training_history.csv'),model_path.split('/')[-1],(len(path_to_compare)-i)/len(path_to_compare)) for i,model_path in enumerate(path_to_compare)]

## Draw performances

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 2)
for history,model_name,i in reversed(histories):
    if plot_training:
        plt.plot(history['loss'], label=f'{model_name}: Training Loss',c=(0,0,1,i))
    if plot_dev:
        plt.plot(history['val_loss'], label=f'{model_name}: Validation Loss',c=(0,0.5,0,i))
    if plot_same_div:
        plt.plot(history['test_same_dist_loss'], label=f'{model_name}: Same Distribution Loss',c=(0.75, 0.0, 0.75,i))
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 1)

for history,model_name,i in reversed(histories):
    if plot_training:
        plt.plot(history['accuracy'], label=f'{model_name}: Training Accuracy',c=(0,0,1,i))
    if plot_dev:
        plt.plot(history['val_accuracy'], label=f'{model_name}: Validation Accuracy',c=(0,0.5,0,i))
    if plot_same_div:
        plt.plot(history['test_same_dist_accuracy'], label=f'{model_name}: Same Distribution Accuracy',c=(0.75, 0.0, 0.75,i))
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()

if smoothened_plot:
    #smothered curve 
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 2)
    for history,model_name,i in reversed(histories):
        if plot_training:
            plt.plot(moving_average(history['loss'],smoothness_level), label=f'{model_name}Training Loss',c=(0,0,1,i))
        if plot_dev:
            plt.plot(moving_average(history['val_loss'],smoothness_level), label=f'{model_name}Validation Loss',c=(0,0.5,0,i))
        if plot_same_div:
            plt.plot(moving_average(history['test_same_dist_loss'],smoothness_level), label=f'{model_name}Same Distribution Loss',c=(0.75, 0.0, 0.75,i))
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 1)
    for history,model_name,i in reversed(histories):
        if plot_training: 
            plt.plot(moving_average(history['accuracy'],smoothness_level), label=f'{model_name}Training Accuracy',c=(0,0,1,i))
        if plot_dev: 
            plt.plot(moving_average(history['val_accuracy'],smoothness_level), label=f'{model_name}Validation Accuracy',c=(0,0.5,0,i))
        if plot_same_div: 
            plt.plot(moving_average(history['test_same_dist_accuracy'],smoothness_level), label=f'{model_name}Same Distribution Accuracy',c=(0.75, 0.0, 0.75,i))
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

plt.show()

epoch_best_result = np.argmax(history['val_accuracy'][max_search_beginning:])+max_search_beginning
print(f'Best result (for the 1st model): (for epoch {epoch_best_result+1})')
print("Accuracy:",history['accuracy'][epoch_best_result] ,"\nLoss:",history['loss'][epoch_best_result],"\nVal_accuracy:",history['val_accuracy'][epoch_best_result], "\nVal_loss",history['val_loss'][epoch_best_result],"\nSame_dist_accuracy",history['test_same_dist_accuracy'][epoch_best_result] if plot_same_div else "","\nSame_dist_loss",history['test_same_dist_loss'][epoch_best_result]  if plot_same_div else "")


## Mean performance

In [ ]:
# return the mean performance of the models matched by the regex expression
models_name = '69-CNN-FT' #regex of the try name
fine_tun= True

train_acc = []
val_acc= []
train_loss = []
val_loss = []
same_dist_acc = []
same_dist_loss = []
try_paths = glob.glob(f'{save_weights_path}/{models_name}')

for try_path in try_paths:
    history =pd.read_csv(f'{try_path}/training_history.csv')

    epoch_best_result = np.argmax(history['val_accuracy'])
    train_acc.append(history['accuracy'][epoch_best_result])
    train_loss.append(history['loss'][epoch_best_result])
    val_acc.append(history['val_accuracy'][epoch_best_result])
    val_loss.append(history['val_loss'][epoch_best_result])

    if not fine_tun:
        same_dist_acc.append(history['test_same_dist_accuracy'][epoch_best_result])
        same_dist_loss.append(history['test_same_dist_loss'][epoch_best_result])

print("Train acc",np.mean(train_acc))
print("Train loss",np.mean(train_loss))
print("Val acc",np.mean(val_acc))
print("Val loss",np.mean(val_loss))
print("Same dist acc",np.mean(same_dist_acc))
print("Same dist loss",np.mean(same_dist_loss))

print("\nVal accuracies: ",val_acc)

# Error analysis

## Load and test model

In [ ]:
# useful to make sure the dataset is the same when running the try twice
def are_tuples_equal(tuple1, tuple2):
    return [np.array_equal(arr1, arr2) for arr1, arr2 in zip(tuple1, tuple2)]

In [ ]:
model_name = '67-LSTM-FT'
model = load_try(model_name)

# Loading the correct datasets
name_person = set(np.unique(target_df['name'])).intersection(model_name.split("-"))
if len(name_person)>0:
    name_person = next(iter(name_person)) # extract name from the string
else:
    name_person = None

data_normal_training,data_fine_tuning,accessory = split_train_dev_test_dataset(test_dev_df_size=200,EMNIST_data_multiplier=8,random_seed=42, person_to_exclude=name_person)

# unpack
train_X, test_same_dist_X, dev_X, test_X, train_Y_categorical, test_same_dist_Y_categorical, dev_Y_categorical, test_Y_categorical = data_normal_training
source_X_train, source_X_test, target_X_train, dev_X, test_X, source_Y_train_categorical, source_Y_test_categorical, target_Y_train_categorical, dev_Y_categorical, test_Y_categorical = data_fine_tuning
train_Y,test_same_dist_Y,source_Y_train,source_Y_test,target_Y_train, dev_Y,test_Y = accessory

In [ ]:
if directML:
    dev_X = dev_X.reshape(dev_X.shape[0],dev_X.shape[1],dev_X.shape[2],1)

In [ ]:
model.evaluate(dev_X, dev_Y_categorical)


In [ ]:
# Predict the labels on the development set
y_pred = model.predict(dev_X)

# Convert one-hot encoded predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert one-hot encoded true labels to class labels
y_dev_labels = np.argmax(dev_Y_categorical, axis=1)

# Find indices of misclassified examples
misclassified_indices = np.where(y_pred_labels != y_dev_labels)[0]
well_classified_indices = np.where(y_pred_labels == y_dev_labels)[0]

desired = []
recognized = []
for i in misclassified_indices:
    desired.append(get_letter_from_index(y_dev_labels[i]+36))
    recognized.append(get_letter_from_index(y_pred_labels[i]+36))

print("number misclassified ", len(misclassified_indices))
print("indices", misclassified_indices)
print("desired", desired)
print("recognized", recognized)


## Explore the mistakes and correct values

### Mistakes

In [ ]:
mistake_number= 10
letter = None #letter or None

In [ ]:
# show misclassified letters - When the cell is rerun, it shows the next mistake
while (letter is not None) and (mistake_number < misclassified_indices.shape[0]-1) and (get_letter_from_index(y_dev_labels[misclassified_indices[mistake_number]]+36) !=letter):
    mistake_number+=1

if np.isin("CNN",network_type):
    plt.imshow(dev_X[misclassified_indices[mistake_number]],"gray")
    # plot_original(dev_X[misclassified_indices[mistake_number]])
elif np.isin("LSTM",network_type):
    X,Y = dev_X[misclassified_indices[mistake_number]].T
    draw_points(X,Y)
plt.title(f'{[misclassified_indices[mistake_number]]}. Desired: {get_letter_from_index(y_dev_labels[misclassified_indices[mistake_number]]+36)} / Recognized: {get_letter_from_index(y_pred_labels[misclassified_indices[mistake_number]]+36)}')


if(mistake_number < misclassified_indices.shape[0]-1):
    mistake_number+=1
else:
    mistake_number=0


### Correct values

In [ ]:
well_classified_number = 0
letter = 'h'

In [ ]:
# show misclassified letters
while (letter is not None) and (well_classified_number < well_classified_indices.shape[0]-1) and (get_letter_from_index(y_dev_labels[well_classified_indices[well_classified_number]]+36) !=letter):
    well_classified_number+=1

if np.isin("CNN",network_type):
    plt.imshow(dev_X[well_classified_indices[well_classified_number]])
    plot_original(dev_X[well_classified_indices[well_classified_number]])
elif np.isin("LSTM",network_type):
    X,Y = dev_X[well_classified_indices[well_classified_number]].T
    draw_points(X,Y)


if(well_classified_number < well_classified_indices.shape[0]-1):
    well_classified_number+=1
else:
    well_classified_number=0

## Draw confusion matrix

In [ ]:
# need to run all previous examples
y_pred_labels = np.argmax(y_pred, axis=1)
y_dev_labels = np.argmax(dev_Y_categorical, axis=1)
letters_labels = [get_letter_from_index(index+36) for index in range(26)]
conf_matrix = confusion_matrix(y_dev_labels, y_pred_labels,labels=np.arange(26))
plt.figure(figsize=(11, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues',xticklabels=letters_labels,yticklabels=letters_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

## Examine examples of a specific set

In [ ]:
letter_number = 0
letter ='d'
set_X = target_X_train
set_Y = target_Y_train # do NOT provide the categorical shape
images = set_X[set_Y==get_index_from_letter(letter)]

In [ ]:

# show every found examples
if np.isin("CNN",network_type):
    plt.imshow(images[letter_number],"gray")
elif np.isin("LSTM",network_type):
    X,Y = images[letter_number].T
    draw_points(X,Y)

if(letter_number < images.shape[0]-1):
    letter_number+=1
else:
    letter_number=0

# Mix the results of 2 different models

In [ ]:
# Attempt to do model fusion to get better overall results

modelA = load_try('30-LSTM')
modelB = load_try('14')

target_index = 111
data =  dev_X[target_index]

valueA = data
valueB = get_image_from_X(data[:32,0],target_df)

draw_points(valueA[:,0],valueA[:,1])
plt.figure()
plt.imshow(valueB)

print("Model A:")
prediction_model_a = (modelA.predict(valueA.reshape(1,250,2)))

print("Model B:")
prediction_model_b = (modelB.predict(valueB.reshape(1,image_shape,image_shape)))

In [ ]:
# observation of the returned results
print(get_letter_from_index(np.argmax(prediction_model_a)+36))
print(get_letter_from_index(np.argmax(prediction_model_b)+36))

In [ ]:
# try to interpret the results
print(get_letter_from_index(np.argmax(prediction_model_a+prediction_model_b)+36))
[get_letter_from_index(a) for a in (np.argsort(np.max(prediction_model_a, axis=0))[::-1]+36)]

# Load Aidar data

In [ ]:
model = load_try('34-LSTM')

In [ ]:
aidar_df = pd.read_csv(f'{absolute_root_path}Previous_work/Personal_Experiments/Dataframes/aidar_interpol1.csv')
total_length = DigiLeTs_X_LSTM.shape[1]

if np.isin("CNN",network_type):
    aidar_df['images'] = aidar_df['images'].apply(ast.literal_eval)
    aidar_df['images'] = aidar_df['images'].apply(np.array)
if np.isin("LSTM",network_type):
    aidar_df['X'] = aidar_df['X'].apply(ast.literal_eval)
    aidar_df['X'] = aidar_df['X'].apply(np.array)
    aidar_df['Y'] = aidar_df['Y'].apply(ast.literal_eval)
    aidar_df['Y'] = aidar_df['Y'].apply(np.array)

aidar_Y = aidar_df['letter'].apply(get_index_from_letter).values # convert letters to numbers
aidar_Y_categorical = to_categorical(aidar_Y-36, num_classes=26)
if np.isin("CNN",network_type):
    aidar_X = np.vstack(aidar_df['images'].values).reshape((1600,image_shape,image_shape))
if np.isin("LSTM",network_type):
    aidar_X_LSTM = np.array([np.vstack(aidar_df['X'].values), np.vstack(aidar_df['Y'].values)]).transpose(1,2,0) # reshape the data to fit the LSTM

    # pad the LSTM value so that it has the same shape that the DigiLeTs values
    aidar_X_LSTM = np.pad(aidar_X_LSTM, ((0, 0), (0, total_length -aidar_X_LSTM.shape[1]), (0, 0)), constant_values=[-10,-10])


In [ ]:
model.evaluate(aidar_X_LSTM,aidar_Y_categorical)

## Explore images from aidar

In [ ]:
image_number = np.random.randint(0,aidar_df.shape[0])
plt.title(get_letter_from_index(aidar_Y[image_number]))
draw_points(aidar_X_LSTM[image_number][:,0],aidar_X_LSTM[image_number][:,1])

## Explore mistakes on Aidar

In [ ]:
# Predict the labels on the development set
y_pred = model.predict(aidar_X_LSTM)

# Convert one-hot encoded predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert one-hot encoded true labels to class labels
y_dev_labels = np.argmax(aidar_Y_categorical, axis=1)

# Find indices of misclassified examples
misclassified_indices = np.where(y_pred_labels != y_dev_labels)[0]

desired = []
recognized = []
for i in misclassified_indices:
    desired.append(get_letter_from_index(y_dev_labels[i]+36))
    recognized.append(get_letter_from_index(y_pred_labels[i]+36))

print("number misclassified ", len(misclassified_indices))
print("indices", misclassified_indices)
print("desired", desired)
print("recognized", recognized)

In [ ]:
letters_total, total_number_letter = np.unique(aidar_df['letter'].values,return_counts=True)
letters_mistake, mistake_number_letter= np.unique(desired,return_counts=True)

In [ ]:
total = {letter: count for letter,count in zip(letters_total, total_number_letter)}
mis = {letter: count for letter,count in zip(letters_mistake, mistake_number_letter)}

In [ ]:
{k: mis[k]/total[k]*100 if  mis[k]/total[k]*100>20 else None for k in mis.keys() & total}

In [ ]:
mistake_number= 0

In [ ]:
if np.isin("CNN",network_type):
    plt.imshow(aidar_X_LSTM[misclassified_indices[mistake_number]])
    plot_original(aidar_X_LSTM[misclassified_indices[mistake_number]])
elif np.isin("LSTM",network_type):
    X,Y = aidar_X_LSTM[misclassified_indices[mistake_number]].T
    draw_points(X,Y)
plt.title(f'{[misclassified_indices[mistake_number]]}. Desired: {get_letter_from_index(y_dev_labels[misclassified_indices[mistake_number]]+36)} / Recognized: {get_letter_from_index(y_pred_labels[misclassified_indices[mistake_number]]+36)}')

if(mistake_number < misclassified_indices.shape[0]-1):
    mistake_number+=1
else:
    mistake_number=0

plt.axis('equal')